# We will Parse and Clean the Advanced Stats Data

In [134]:
from bs4 import BeautifulSoup
import pandas as pd

In [135]:
seasons = list(range(1996, 2023))

In [136]:
dfs = []

# looping throuch each season and appending each advanced stats table to one big dataframe

for season in seasons:
    file_name = "ADVANCED/{}_adv_stats.html".format(season)

    with open(file_name) as f:
        adv_stats_table = f.read()

    soup = BeautifulSoup(adv_stats_table, "html.parser")

    # find html elements that need to removed (the headers)
    over_header = soup.find(class_="over_header")
    over_header_thead = soup.find(class_="over_header thead")
    thead = soup.find(class_="thead")

    # remove those html elements
    if over_header:
        over_header.decompose()
    
    if over_header_thead:
        over_header_thead.decompose()
    
    if thead:
        thead.decompose()

    # turn the html into a pandas dataframe
    adv_stats = pd.read_html(str(soup))[0]

    # removing the thead element because apparently the previous if statement doesn't work
    # this will remove the rows that have the value "MOV" under the MOV column (which are the thead rows)
    adv_stats = adv_stats[adv_stats["MOV"] != "MOV"]

    # dropping the "League Average Row" in the dataframe (which is the last row)
    adv_stats = adv_stats.drop(adv_stats.index[-1])

    # adding a year column to the dataframe
    adv_stats["Year"] = season

    # add the dataframe to a list
    dfs.append(adv_stats)

# join all the dataframes into one big dataframe
adv_df = pd.concat(dfs)

In [137]:
adv_df

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,Unnamed: 22,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Unnamed: 27,Arena,Attend.,Attend./G,Year
0,1.0,Chicago Bulls*,29.9,72.0,10.0,70,12,12.24,-0.44,11.8,...,NaN,0.482,16.1,71.1,0.222,NaN,United Center,969149,23638,1996
1,2.0,Seattle SuperSonics*,29.6,64.0,18.0,61,21,7.79,-0.4,7.4,...,NaN,0.479,16.7,69.5,0.252,NaN,KeyArena at Seattle Center,697301,17007,1996
2,3.0,Utah Jazz*,29.5,55.0,27.0,59,23,6.59,-0.34,6.25,...,NaN,0.489,15.1,71.7,0.295,NaN,Delta Center,813073,19831,1996
3,4.0,San Antonio Spurs*,29.4,59.0,23.0,58,24,6.3,-0.33,5.98,...,NaN,0.472,13.9,70.0,0.216,NaN,Alamodome,782701,19791,1996
4,5.0,Orlando Magic*,27.7,60.0,22.0,56,26,5.56,-0.16,5.4,...,NaN,0.491,14.0,68.8,0.223,NaN,Orlando Arena,707168,17248,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,26,Detroit Pistons,23.6,23,59,22,60,-7.72,0.37,-7.36,...,NaN,.541,13.1,75.6,.226,NaN,Little Caesars Arena,663556,16184,2022
27,27,Orlando Magic,23.3,22,60,21,61,-8.00,0.33,-7.67,...,NaN,.532,11.7,77.2,.196,NaN,Amway Center,622881,15192,2022
28,28,Oklahoma City Thunder,22.4,24,58,21,61,-8.10,0.20,-7.90,...,NaN,.533,11.8,76.1,.169,NaN,Paycom Center,595112,14515,2022
29,29,Houston Rockets,24.1,20,62,21,61,-8.48,0.22,-8.26,...,NaN,.554,12.3,74.4,.206,NaN,Toyota Center,638977,15585,2022


In [138]:
# deleting unnecessary columns

del adv_df["Rk"]
del adv_df["Age"]
del adv_df["W"]
del adv_df["PW"]
del adv_df["PL"]
del adv_df["L"]
del adv_df["Arena"]
del adv_df["Attend."]
del adv_df["Attend./G"]
del adv_df["Unnamed: 17"]
del adv_df["Unnamed: 22"]
del adv_df["Unnamed: 27"]

In [139]:
adv_df

,Team,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Year
0,Chicago Bulls*,12.24,-0.44,11.8,115.2,101.8,13.4,91.1,0.291,0.196,0.555,0.517,13.1,36.9,0.217,0.482,16.1,71.1,0.222,1996
1,Seattle SuperSonics*,7.79,-0.4,7.4,110.3,102.1,8.2,93.8,0.379,0.249,0.574,0.526,16.2,29.7,0.288,0.479,16.7,69.5,0.252,1996
2,Utah Jazz*,6.59,-0.34,6.25,113.3,106.1,7.2,90.0,0.359,0.158,0.566,0.517,14.1,31.6,0.276,0.489,15.1,71.7,0.295,1996
3,San Antonio Spurs*,6.3,-0.33,5.98,110.2,103.5,6.7,93.3,0.342,0.2,0.558,0.516,13.6,27.5,0.252,0.472,13.9,70.0,0.216,1996
4,Orlando Magic*,5.56,-0.16,5.4,112.9,106.9,6.0,91.8,0.336,0.248,0.562,0.529,13.2,29.0,0.232,0.491,14.0,68.8,0.223,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,Detroit Pistons,-7.72,0.37,-7.36,106.0,113.8,-7.8,98.4,.249,.391,.533,.494,12.6,23.4,.194,.541,13.1,75.6,.226,2022
27,Orlando Magic,-8.00,0.33,-7.67,104.5,112.5,-8.0,99.2,.223,.417,.538,.503,13.0,19.8,.175,.532,11.7,77.2,.196,2022
28,Oklahoma City Thunder,-8.10,0.20,-7.90,104.6,112.8,-8.2,98.5,.223,.419,.530,.497,12.5,21.8,.169,.533,11.8,76.1,.169,2022
29,Houston Rockets,-8.48,0.22,-8.26,108.4,116.7,-8.3,100.9,.284,.448,.565,.534,14.5,21.7,.202,.554,12.3,74.4,.206,2022


**Renaming the columns that end in .1 to _opp (indicating opponent data)**

In [140]:
adv_df.columns = adv_df.columns.str.replace(".1", "_opp")
adv_df

,Team,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%_opp,TOV%_opp,DRB%,FT/FGA_opp,Year
0,Chicago Bulls*,12.24,-0.44,11.8,115.2,101.8,13.4,91.1,0.291,0.196,0.555,0.517,13.1,36.9,0.217,0.482,16.1,71.1,0.222,1996
1,Seattle SuperSonics*,7.79,-0.4,7.4,110.3,102.1,8.2,93.8,0.379,0.249,0.574,0.526,16.2,29.7,0.288,0.479,16.7,69.5,0.252,1996
2,Utah Jazz*,6.59,-0.34,6.25,113.3,106.1,7.2,90.0,0.359,0.158,0.566,0.517,14.1,31.6,0.276,0.489,15.1,71.7,0.295,1996
3,San Antonio Spurs*,6.3,-0.33,5.98,110.2,103.5,6.7,93.3,0.342,0.2,0.558,0.516,13.6,27.5,0.252,0.472,13.9,70.0,0.216,1996
4,Orlando Magic*,5.56,-0.16,5.4,112.9,106.9,6.0,91.8,0.336,0.248,0.562,0.529,13.2,29.0,0.232,0.491,14.0,68.8,0.223,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,Detroit Pistons,-7.72,0.37,-7.36,106.0,113.8,-7.8,98.4,.249,.391,.533,.494,12.6,23.4,.194,.541,13.1,75.6,.226,2022
27,Orlando Magic,-8.00,0.33,-7.67,104.5,112.5,-8.0,99.2,.223,.417,.538,.503,13.0,19.8,.175,.532,11.7,77.2,.196,2022
28,Oklahoma City Thunder,-8.10,0.20,-7.90,104.6,112.8,-8.2,98.5,.223,.419,.530,.497,12.5,21.8,.169,.533,11.8,76.1,.169,2022
29,Houston Rockets,-8.48,0.22,-8.26,108.4,116.7,-8.3,100.9,.284,.448,.565,.534,14.5,21.7,.202,.554,12.3,74.4,.206,2022


**Removing the Stars from the Team Name and Renaming the Team Name to the Abbrev**

In [141]:
adv_df["Team"] = adv_df["Team"].str.replace("*", "", regex=False)

In [142]:
# Creating a dictionary from the team_abbrev csv file where the keys are the team's full name
# and the values are the abbreviated name

team_abbrev = {}

with open("CSV/team_abbrev.csv") as file:
    entries = file.readlines()
    for entry in entries[1:]:
        abbrev, full_name = entry.replace("\n", "").split(",")
        team_abbrev[full_name] = abbrev

In [143]:
adv_df["Team"] = adv_df["Team"].map(team_abbrev)

In [144]:
adv_df

,Team,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%_opp,TOV%_opp,DRB%,FT/FGA_opp,Year
0,CHI,12.24,-0.44,11.8,115.2,101.8,13.4,91.1,0.291,0.196,0.555,0.517,13.1,36.9,0.217,0.482,16.1,71.1,0.222,1996
1,OKC,7.79,-0.4,7.4,110.3,102.1,8.2,93.8,0.379,0.249,0.574,0.526,16.2,29.7,0.288,0.479,16.7,69.5,0.252,1996
2,UTA,6.59,-0.34,6.25,113.3,106.1,7.2,90.0,0.359,0.158,0.566,0.517,14.1,31.6,0.276,0.489,15.1,71.7,0.295,1996
3,SAS,6.3,-0.33,5.98,110.2,103.5,6.7,93.3,0.342,0.2,0.558,0.516,13.6,27.5,0.252,0.472,13.9,70.0,0.216,1996
4,ORL,5.56,-0.16,5.4,112.9,106.9,6.0,91.8,0.336,0.248,0.562,0.529,13.2,29.0,0.232,0.491,14.0,68.8,0.223,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,DET,-7.72,0.37,-7.36,106.0,113.8,-7.8,98.4,.249,.391,.533,.494,12.6,23.4,.194,.541,13.1,75.6,.226,2022
27,ORL,-8.00,0.33,-7.67,104.5,112.5,-8.0,99.2,.223,.417,.538,.503,13.0,19.8,.175,.532,11.7,77.2,.196,2022
28,OKC,-8.10,0.20,-7.90,104.6,112.8,-8.2,98.5,.223,.419,.530,.497,12.5,21.8,.169,.533,11.8,76.1,.169,2022
29,HOU,-8.48,0.22,-8.26,108.4,116.7,-8.3,100.9,.284,.448,.565,.534,14.5,21.7,.202,.554,12.3,74.4,.206,2022


In [145]:
adv_df.to_csv("CSV/advanced_stats.csv")